# Import librairies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings("ignore") # to avoid deprecation warnings


In [2]:
!pwd

/home/jovyan/fullstack/PROJET UBER


In [2]:
#Explore dataset: two types
## Explorer un ensemble de données : deux types 



df_pickup_raw=pd.read_csv("uber-raw-data-apr14.csv")

print(df_pickup_raw.shape)
df_pickup_raw.head()

(564516, 4)


,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


# EXPLORING PICKUP

In [3]:
df_pickup=df_pickup_raw.copy()
df_pickup

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
564511,4/30/2014 23:22:00,40.7640,-73.9744,B02764
564512,4/30/2014 23:26:00,40.7629,-73.9672,B02764
564513,4/30/2014 23:31:00,40.7443,-73.9889,B02764
564514,4/30/2014 23:32:00,40.6756,-73.9405,B02764


In [4]:
df_pickup['date']=pd.to_datetime(df_pickup['Date/Time'], format='%m/%d/%Y %H:%M:%S', errors='ignore')
df_pickup.head()

,Date/Time,Lat,Lon,Base,date
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,2014-04-01 00:11:00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,2014-04-01 00:17:00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,2014-04-01 00:21:00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,2014-04-01 00:28:00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,2014-04-01 00:33:00


In [6]:
# en fait il faut creer et rajouter les nouvelles colonnes date a notre dataframe

df_pickup=df_pickup_raw.copy()

df_pickup['date']=pd.to_datetime(df_pickup['Date/Time'], format='%m/%d/%Y %H:%M:%S', errors='ignore')
df_pickup['day']=df_pickup.date.dt.day
df_pickup['month'] = df_pickup.date.dt.month
df_pickup['hour'] = df_pickup.date.dt.hour
df_pickup['dayofweek'] = df_pickup.date.dt.dayofweek
df_pickup['quarter'] = df_pickup.date.dt.quarter
df_pickup['dayofyear'] = df_pickup.date.dt.dayofyear
df_pickup['dayofmonth'] = df_pickup.date.dt.day
df_pickup['weekofyear'] = df_pickup.date.dt.isocalendar().week

df_pickup.head()


,Date/Time,Lat,Lon,Base,date,day,month,hour,dayofweek,quarter,dayofyear,dayofmonth,weekofyear
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,2014-04-01 00:11:00,1,4,0,1,2,91,1,14
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,2014-04-01 00:17:00,1,4,0,1,2,91,1,14
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,2014-04-01 00:21:00,1,4,0,1,2,91,1,14
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,2014-04-01 00:28:00,1,4,0,1,2,91,1,14
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,2014-04-01 00:33:00,1,4,0,1,2,91,1,14


# Analyse exploratoire

# Effectuons un k-means sur Lat et Lon pour les afficher sur une carte avec des couleurs par cluster.
## Cela nous permettra de voir comment les données sont regroupées avant de procéder à toute autre analyse.

In [7]:
# Import K-Means 
from sklearn.cluster import KMeans

X=df_pickup[['Lat','Lon']]

# Puisque les données sont volumineuses, prenons un échantillon pour le tester(10000)

X_sample=X.sample(n=10000)



# Kmeans: Nous testons différents nombres de cluster et visualisons sur la carte
# pour voir plus ou moins quel est le meilleur k
# le meilleur ici c'est le 6

kmeans = KMeans(n_clusters=6, random_state=0, init="k-means++")

#Fit pour adapter KMeans à notre dataset
kmeans.fit(X_sample)

## Obtenir le cluster 
# on rajoute une colonne dans notre dataframe X_sample qui s'appelle cluster qui sera la prédiction

X_sample['cluster'] = kmeans.predict(X_sample)
X_sample.head()

,Lat,Lon,cluster
358082,40.7295,-73.9899,1
320145,40.7314,-73.9885,1
58072,40.7384,-73.9855,1
152321,40.7227,-73.9982,1
404908,40.7698,-73.8643,0


In [8]:
X_sample['cluster'].value_counts()

cluster
1    4452
5    3780
4    1077
0     349
2     253
3      89
Name: count, dtype: int64

In [10]:
!pip install plotly

In [10]:
#Visualize number of clusters on map
#Visualiser le nombre de clusters sur la carte

import plotly.express as px

fig = px.scatter_mapbox(X_sample, lat="Lat", lon="Lon", color="cluster", zoom=10, mapbox_style="carto-positron")
fig.show()

#### voir si c'est différent selon les jours de la semaine 
#### on va filtrer par jour et on va clusteriser pour voir les changements en fonction des jours

# Explorer le clustering basé sur certaines caractéristiques de date

In [11]:
#Display days of week
pd.unique(df_pickup['dayofweek'])

array([1, 2, 3, 4, 5, 6, 0])

In [12]:
#Days of week
#Le jour de la semaine avec lundi=0, dimanche=6.

daysweek=pd.unique(df_pickup['dayofweek']).tolist()# pour avoir une liste

type(daysweek) # pour vérifier que c'est bien une liste
daysweek.sort()
daysweek

[0, 1, 2, 3, 4, 5, 6]

# Test avec l'algorithme Kmeans

 voir si c'est différent selon les jours de la semaine 
 on peut faire une boucle  0 à 6 pour voir si Lundi est différent du mardi ...etc 
 on va filtrer par jour et on va clusteriser pour voir les changements en fonction des jours

## mais on va faire plutôt une fonction comme ça on peut lui demander de ploter

### On filtre par dayofweek( jour de la semaine)

In [13]:
filt_val='dayofweek'
df_pickup[["Lat","Lon", filt_val ]]

,Lat,Lon,dayofweek
0,40.7690,-73.9549,1
1,40.7267,-74.0345,1
2,40.7316,-73.9873,1
3,40.7588,-73.9776,1
4,40.7594,-73.9722,1
...,...,...,...
564511,40.7640,-73.9744,2
564512,40.7629,-73.9672,2
564513,40.7443,-73.9889,2
564514,40.6756,-73.9405,2


In [14]:
def kmeans_fn(day,kval,df=df_pickup,samp=10000):
    X_sample=df.loc[df["dayofweek"]==day,["Lat", "Lon"]].sample(n=samp)
    
    ##kmeans : nous testons différents nombres de cluster et les visualisons sur la carte pour voir plus ou moins quel est le meilleur k
    
    kmeans = KMeans(n_clusters=kval, random_state=0, init="k-means++")
    #Fit pour adapter KMeans à notre dataset
    kmeans.fit(X_sample)
    ## Obtenir le cluster 
    ## on rajoute une colonne dans notre dataframe X_sample qui s'appelle cluster qui sera la prédiction
    X_sample['cluster'] = kmeans.predict(X_sample)
    ## on visualise
    fig = px.scatter_mapbox(X_sample, lat="Lat", lon="Lon", color='cluster', zoom=10, mapbox_style="carto-positron",title=day)
    fig.show("iframe")

In [15]:
kmeans_fn(5,kval=6,samp=10000)# Pour samedi

In [17]:
kmeans_fn(6,kval=6,samp=10000)# Pour dimanche

In [16]:
# on fait une boucle pour les avoir pour tous les jours
for i in daysweek:
   kmeans_fn(i,kval=6,samp=10000) 

# Test avec DBSCAN

In [17]:
#With All days of week
#Avec tous les jours de la semaine

from sklearn.cluster import DBSCAN

X_sample=df_pickup[["Lat","Lon"]].sample(n=20000)

#DBSCAN:

# Instanciate DBSCAN 
db = DBSCAN(eps=0.01, min_samples=100, metric="manhattan", algorithm="auto")
#Fit DBSCAN to our dataset
db.fit(X_sample)
## Get cluster 
X_sample['cluster'] = db.labels_
#Number of found clusters
X_sample['cluster'].nunique()

6

### En jouant sur la distance eps (0.001, 0.01) et sur la taille de sample (10000, 20000) on trouve plus ou moins de clusters.

In [18]:
X_sample['cluster'].value_counts()

cluster
 1    16766
-1     1783
 2      601
 0      408
 3      313
 4      129
Name: count, dtype: int64

In [20]:
#Visualize
fig = px.scatter_mapbox(X_sample, lat="Lat", lon="Lon", color="cluster", zoom=10, mapbox_style="carto-positron",title="All days of week")
fig.show()

### DBSCAN

#### IL REPERE MANHATTAN ET BROOKLYN  cluster 0 
#### LA OU IL Y'A VRAIMENT BEAUCOUP DE MONDE ET C'EST LA QU'ON A ENVIE DE METTRE BEAUCOUP DE TAXIS
#### Apres il repere le petit cluster en orange LE QUEENS
Et le reste c'est un peu plus éparse

## il faudrait peut être augmenter un peu la distance pour avoir moins d'outliers


### kmeans 
voir graphique 6:

 * A du diviser manathann en 2 alors que ce n'est pas necéssaire (en  orange et en violet)
 * parce que manathann a une forme horizentale étendue.


### ça parait plus cohérent de faire un cluster où on reste sur manathann comme le fait DBSCAN